In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

tf.executing_eagerly()

c:\users\windows\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.0.0
Default GPU Device: /device:GPU:0


True

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

In [3]:
data = np.load('./helpers/data.npy',allow_pickle=True)

In [4]:
my_dict = dict()
for d in data:
    if d.shape[0] not in my_dict:
        my_dict[d.shape[0]] = []
    my_dict[d.shape[0]].append(d)
    
for k in my_dict:
    my_dict[k] = np.stack(my_dict[k])
    print(my_dict[k].shape)

(3, 4, 4)
(2, 5, 4)
(7, 7, 4)
(43, 8, 4)
(35, 9, 4)
(18, 10, 4)
(228, 11, 4)
(506, 12, 4)
(628, 13, 4)
(431, 14, 4)
(1870, 15, 4)
(4256, 16, 4)
(7292, 17, 4)
(5418, 19, 4)
(37133, 20, 4)
(42408, 23, 4)


In [5]:
indexes = np.argmax(my_dict[23][0],axis=1)
indexes

array([1, 1, 1, 2, 1, 0, 2, 1, 1, 0, 2, 0, 0, 0, 2, 0, 1, 0, 2, 0, 2, 0,
       0], dtype=int64)

In [6]:
from helpers.cfg import CFGHelper
from helpers.dataset_generator import sample_grammar

cfg = CFGHelper(sample_grammar)
tokens = cfg.indexes_to_tokens(indexes)
' '.join(tokens)

'( x + x ) + x + x + x - y'

In [7]:
batch_size = 5
lstm_units = 100
vocab_size = 4
seq_len = 20

In [8]:
class CFGNetEncoder(tf.keras.layers.Layer):
    def __init__(self, lstm_units, vae_std_dev=1e-2):
        super(CFGNetEncoder, self).__init__()
        
        self.lstm_units = lstm_units
        self.recurrent = tf.keras.layers.LSTM(lstm_units,return_state=True)
        self.dense1 = tf.keras.layers.Dense(lstm_units * 2)
        self.dense2 = tf.keras.layers.Dense(lstm_units * 2)
        self.vae_std_dev = vae_std_dev

    def build(self, input_shape):
        super(CFGNetEncoder, self).build(input_shape)

    def _sample(self, z_mean, z_log_sigma):
        batch_size = tf.shape(z_mean)[0]
        latent_dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch_size, latent_dim),stddev=self.vae_std_dev)
        return z_mean + tf.keras.backend.exp(z_log_sigma) * epsilon
    
    def call(self, v):
        _,s1,s2 = self.recurrent(v)
        out = tf.concat([s1,s2],axis=1)
        z_mean = self.dense1(out)
        z_log_sigma = self.dense2(out)
        return self._sample(z_mean, z_log_sigma)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.latent_dim)


In [9]:
# temp_encoder = CFGNetEncoder(lstm_units)
# x = tf.random.normal((batch_size, seq_len, vocab_size))
# y = tf.random.normal((batch_size, lstm_units * 2))
# state = temp_encoder(x)
# state.shape
# print(len(temp_encoder.trainable_variables))

# a = Input(shape=(lstm_units, vocab_size))
# b = temp_encoder(a)
# m = Model(inputs=a, outputs=b)
# m.compile(loss='mse', optimizer='adam')
# m.fit(x=x,y=y,epochs=100,batch_size=batch_size)

In [10]:
class CFGNetDecoder(tf.keras.layers.Layer):
    def __init__(self, lstm_units,seq_len, vocab_size):   
        super(CFGNetDecoder, self).__init__()
        
        self.lstm_units = lstm_units
        self.vocab_size = vocab_size
        self.seq_len = seq_len  # TODO: Find a more elegant way of doing this
        self.recurrent = tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        self.dense1 = tf.keras.layers.Dense(lstm_units)
        self.dense2 = tf.keras.layers.Dense(vocab_size, activation='softmax')
    
    def build(self, input_shape):
        super(CFGNetDecoder, self).build(input_shape)
        
    def call(self, state):
        states = tf.split(state, num_or_size_splits=2, axis=1)
        hidden_state_shape = tf.shape(states[0])
        batch_size = hidden_state_shape[0]
        vocab_size = self.vocab_size
        v = tf.expand_dims(state,1)
        v = tf.tile(v, [1, self.seq_len, 1])
        out = self.recurrent(v, initial_state=states)
        return self.dense2(out)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.seq_len, self.vocab_size)

In [11]:
# temp_decoder = CFGNetDecoder(lstm_units,seq_len, vocab_size)
# y = tf.random.normal((batch_size, seq_len, vocab_size))
# x = tf.random.normal((batch_size, lstm_units * 2))
# out = temp_decoder(x)
# out.shape
# # print(temp_decoder.trainable_variables)

# a = Input(shape=(lstm_units * 2))
# b = temp_decoder(a)
# m = Model(inputs=a, outputs=b)
# m.compile(loss='mse', optimizer='adam')
# m.fit(x=x,y=y,epochs=100,batch_size=batch_size)

In [12]:
class CFGNetAutoencoder(tf.keras.Model):
    def __init__(self, lstm_units,seq_len,vocab_size):
        super(CFGNetAutoencoder, self).__init__()
        
        self.lstm_units = lstm_units
        self.seq_len = seq_len # TODO: Find a more elegant way of doing this
        self.vocab_size = vocab_size
        self.encoder = CFGNetEncoder(lstm_units)
        self.decoder = CFGNetDecoder(lstm_units, seq_len, vocab_size)

    def build(self, input_shape):
        super(CFGNetAutoencoder, self).build(input_shape)
        
    def call(self, v):
        state = self.encoder(v)
        return self.decoder(state)
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [13]:
# temp_ae = CFGNetAutoencoder(lstm_units,seq_len, vocab_size)
# x = tf.random.normal((batch_size, seq_len, vocab_size))
# y = x
# out = temp_ae(x)
# temp_ae.compile(loss='categorical_crossentropy', optimizer='adam')
# temp_ae.fit(x=x,y=y,epochs=100,batch_size=batch_size)

In [14]:
seq_len = 17

In [15]:
data_to_use = my_dict[seq_len]
data_to_use.shape

(7292, 17, 4)

In [16]:
cfg_net_autoencoder = CFGNetAutoencoder(lstm_units,seq_len, vocab_size)
cfg_net_autoencoder.compile(loss='categorical_crossentropy', optimizer='adam')
cfg_net_autoencoder(tf.zeros((batch_size, seq_len, vocab_size)))
cfg_net_autoencoder.summary()

Model: "cfg_net_autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cfg_net_encoder (CFGNetEncod multiple                  122400    
_________________________________________________________________
cfg_net_decoder (CFGNetDecod multiple                  120804    
Total params: 243,204
Trainable params: 243,204
Non-trainable params: 0
_________________________________________________________________


In [17]:
from datetime import datetime
# for key in my_dict:
# for key in [seq_len]:
data_to_use = my_dict[seq_len]
logdir = "logs\\adam\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tbCallback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=0,profile_batch=0)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=2e-2,
                          patience=5, min_lr=1e-9, mode='min')
cb_nan = tf.keras.callbacks.TerminateOnNaN()
cfg_net_autoencoder.fit(x=data_to_use,y=data_to_use,batch_size=4000,epochs=5000, callbacks=[tbCallback,reduce_lr,cb_nan])

import winsound
frequency = 2500 
duration = 500 
winsound.Beep(frequency, duration)

Train on 7292 samples
Epoch 1/5000
7292/7292 [==============================] - 7s 907us/sample - loss: 1.3499
Epoch 2/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 1.2830
Epoch 3/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 1.2649
Epoch 4/5000
7292/7292 [==============================] - 1s 135us/sample - loss: 1.2398
Epoch 5/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 1.2132
Epoch 6/5000
7292/7292 [==============================] - 1s 137us/sample - loss: 1.1895
Epoch 7/5000
7292/7292 [==============================] - 1s 134us/sample - loss: 1.1659
Epoch 8/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 1.1529
Epoch 9/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 1.1441
Epoch 10/5000
7292/7292 [==============================] - 1s 134us/sample - loss: 1.1288
Epoch 11/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 1.11

7292/7292 [==============================] - 1s 128us/sample - loss: 0.7080
Epoch 92/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 0.7069
Epoch 93/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 0.7058
Epoch 94/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 0.7048
Epoch 95/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 0.7038
Epoch 96/5000
7292/7292 [==============================] - 1s 127us/sample - loss: 0.7030
Epoch 97/5000
7292/7292 [==============================] - 1s 128us/sample - loss: 0.7022
Epoch 98/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 0.7014
Epoch 99/5000
7292/7292 [==============================] - 1s 150us/sample - loss: 0.7007
Epoch 100/5000
7292/7292 [==============================] - 1s 135us/sample - loss: 0.6999
Epoch 101/5000
7292/7292 [==============================] - 1s 140us/sample - loss: 0.6991
Epoch 102/5000
7292/72

7292/7292 [==============================] - 1s 131us/sample - loss: 0.6730
Epoch 182/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 0.6728
Epoch 183/5000
7292/7292 [==============================] - 1s 130us/sample - loss: 0.6725
Epoch 184/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.6723
Epoch 185/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.6720
Epoch 186/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 0.6718
Epoch 187/5000
7292/7292 [==============================] - 1s 135us/sample - loss: 0.6716
Epoch 188/5000
7292/7292 [==============================] - 1s 136us/sample - loss: 0.6713
Epoch 189/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 0.6711
Epoch 190/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 0.6708
Epoch 191/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.6706
Epoch 192/5000

7292/7292 [==============================] - 1s 147us/sample - loss: 0.6522
Epoch 272/5000
7292/7292 [==============================] - 1s 143us/sample - loss: 0.6519
Epoch 273/5000
7292/7292 [==============================] - 1s 136us/sample - loss: 0.6517
Epoch 274/5000
7292/7292 [==============================] - 1s 146us/sample - loss: 0.6515
Epoch 275/5000
7292/7292 [==============================] - 1s 153us/sample - loss: 0.6512
Epoch 276/5000
7292/7292 [==============================] - 1s 142us/sample - loss: 0.6510
Epoch 277/5000
7292/7292 [==============================] - 1s 154us/sample - loss: 0.6508
Epoch 278/5000
7292/7292 [==============================] - 1s 141us/sample - loss: 0.6506
Epoch 279/5000
7292/7292 [==============================] - 1s 137us/sample - loss: 0.6503
Epoch 280/5000
7292/7292 [==============================] - 1s 142us/sample - loss: 0.6501
Epoch 281/5000
7292/7292 [==============================] - 1s 147us/sample - loss: 0.6499
Epoch 282/5000

7292/7292 [==============================] - 1s 133us/sample - loss: 0.6316
Epoch 362/5000
7292/7292 [==============================] - 1s 130us/sample - loss: 0.6313
Epoch 363/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.6311
Epoch 364/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 0.6309
Epoch 365/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 0.6306
Epoch 366/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.6304
Epoch 367/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.6302
Epoch 368/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 0.6299
Epoch 369/5000
7292/7292 [==============================] - 1s 129us/sample - loss: 0.6297
Epoch 370/5000
7292/7292 [==============================] - 1s 128us/sample - loss: 0.6295
Epoch 371/5000
7292/7292 [==============================] - 1s 135us/sample - loss: 0.6293
Epoch 372/5000

...

7292/7292 [==============================] - 1s 134us/sample - loss: 0.1643
Epoch 3128/5000
7292/7292 [==============================] - 1s 134us/sample - loss: 0.1643
Epoch 3129/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoch 3130/5000
7292/7292 [==============================] - 1s 135us/sample - loss: 0.1643
Epoch 3131/5000
7292/7292 [==============================] - 1s 140us/sample - loss: 0.1643
Epoch 3132/5000
7292/7292 [==============================] - 1s 139us/sample - loss: 0.1643
Epoch 3133/5000
7292/7292 [==============================] - 1s 139us/sample - loss: 0.1643
Epoch 3134/5000
7292/7292 [==============================] - 1s 134us/sample - loss: 0.1643
Epoch 3135/5000
7292/7292 [==============================] - 1s 186us/sample - loss: 0.1643
Epoch 3136/5000
7292/7292 [==============================] - 1s 138us/sample - loss: 0.1643
Epoch 3137/5000
7292/7292 [==============================] - 1s 138us/sample - loss: 0.1643
Epoc

7292/7292 [==============================] - 1s 138us/sample - loss: 0.1643
Epoch 3217/5000
7292/7292 [==============================] - 1s 136us/sample - loss: 0.1643
Epoch 3218/5000
7292/7292 [==============================] - 1s 134us/sample - loss: 0.1643
Epoch 3219/5000
7292/7292 [==============================] - 1s 130us/sample - loss: 0.1643
Epoch 3220/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoch 3221/5000
7292/7292 [==============================] - 1s 131us/sample - loss: 0.1643
Epoch 3222/5000
7292/7292 [==============================] - 1s 134us/sample - loss: 0.1643
Epoch 3223/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoch 3224/5000
7292/7292 [==============================] - 1s 135us/sample - loss: 0.1643
Epoch 3225/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoch 3226/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoc

7292/7292 [==============================] - 1s 130us/sample - loss: 0.1643
Epoch 3306/5000
7292/7292 [==============================] - 1s 144us/sample - loss: 0.1643
Epoch 3307/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.1643
Epoch 3308/5000
7292/7292 [==============================] - 1s 139us/sample - loss: 0.1643
Epoch 3309/5000
7292/7292 [==============================] - 1s 144us/sample - loss: 0.1643
Epoch 3310/5000
7292/7292 [==============================] - 1s 137us/sample - loss: 0.1643
Epoch 3311/5000
7292/7292 [==============================] - 1s 145us/sample - loss: 0.1643
Epoch 3312/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoch 3313/5000
7292/7292 [==============================] - 1s 140us/sample - loss: 0.1643
Epoch 3314/5000
7292/7292 [==============================] - 1s 193us/sample - loss: 0.1643
Epoch 3315/5000
7292/7292 [==============================] - 1s 144us/sample - loss: 0.1643
Epoc

7292/7292 [==============================] - 1s 141us/sample - loss: 0.1643
Epoch 3395/5000
7292/7292 [==============================] - 1s 134us/sample - loss: 0.1643
Epoch 3396/5000
7292/7292 [==============================] - 1s 135us/sample - loss: 0.1643
Epoch 3397/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoch 3398/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoch 3399/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.1643
Epoch 3400/5000
7292/7292 [==============================] - 1s 137us/sample - loss: 0.1643
Epoch 3401/5000
7292/7292 [==============================] - 1s 144us/sample - loss: 0.1643
Epoch 3402/5000
7292/7292 [==============================] - 1s 137us/sample - loss: 0.1643
Epoch 3403/5000
7292/7292 [==============================] - 1s 133us/sample - loss: 0.1643
Epoch 3404/5000
7292/7292 [==============================] - 1s 132us/sample - loss: 0.1643
Epoc

KeyboardInterrupt: 

In [18]:
cfg_net_autoencoder.save_weights('./good_boi.h5')

In [19]:
predictions = cfg_net_autoencoder.predict(data_to_use)

In [26]:
y_ = np.argmax(data_to_use,axis=2)
y_

array([[1, 1, 0, ..., 2, 0, 0],
       [1, 1, 0, ..., 2, 0, 0],
       [1, 1, 1, ..., 2, 0, 0],
       ...,
       [1, 1, 1, ..., 3, 0, 2],
       [1, 1, 1, ..., 3, 0, 2],
       [1, 1, 2, ..., 3, 0, 2]], dtype=int64)

In [24]:
y = np.argmax(predictions,axis=2)
y

array([[1, 1, 0, ..., 2, 0, 0],
       [1, 1, 0, ..., 2, 0, 0],
       [1, 1, 1, ..., 2, 0, 0],
       ...,
       [1, 1, 1, ..., 3, 0, 2],
       [1, 1, 1, ..., 3, 0, 2],
       [1, 1, 2, ..., 3, 0, 2]], dtype=int64)

In [32]:
for i in range(0,0+min(y.shape[0],50)):
    try:
        t1 = cfg.indexes_to_tokens(y_[i])
        t2 = cfg.indexes_to_tokens(y[i])
        print(''.join(t1),'::',''.join(t2))
    except Exception as e:
        print(e)

((x))+(x)+x+x :: ((x))+(x)+x+x
((x))+(x)+x+w :: ((x))+(x)+x+w
((x))+(x)+x-z :: ((x))+(x)+x-z
((x))+(x)+x*y :: ((x))+(x)+x*x
((x))+(x)+z*x :: ((x))+(x)+z*x
((x))+(x)+z*y :: ((x))*(x)+z*x
((x))+(x)+z/w :: ((x))+(x)+z/x
((x))+(x)+w*x :: ((x))+(x)+w*x
((x))+(x)+w/y :: ((x))/(x)+w/x
((x))+(x)-x+w :: ((x))+(x)-x+w
((x))+(x)-x*y :: ((x))+(x)-x*y
((x))+(x)-y*w :: ((x))-(x)-y*w
((x))+(x)-z*z :: ((x))*(x)-z*z
((x))+(x)-w+z :: ((x))+(x)-w+z
((x))+(x)-w*z :: ((x))/(x)-w*z
((x))+(x)-w/w :: ((x))/(x)/w/w
((x))+(x)*x*x :: ((x))+(x)*x*x
((x))+(x)*x*z :: ((x))+(x)*x*z
((x))+(x)*y+z :: ((x))+(x)*y+z
((x))+(x)*z+x :: ((x))+(x)*z+x
((x))+(x)*z+z :: ((x))*(x)*z+z
((x))+(x)*z-y :: ((x))+(x)*z-y
((x))+(x)*w+y :: ((x))+(x)*x+y
((x))+(x)*w*x :: ((x))+(x)*w*x
((x))+(x)/x+y :: ((x))+(x)/x+y
((x))+(x)/x-y :: ((x))+(x)/x-y
((x))+(x)/x-z :: ((x))+(x)/x-z
((x))+(x)/y-y :: ((x))-(x)/y-y
((x))+(x)/z*w :: ((x))+(x)/z*w
((x))+(x)/w+x :: ((x))/(x)+x+w
((x))+(y)+x+y :: ((x))+(x)+x+x
((x))+(y)+x-w :: ((x))+(x)+x-w
((x))+(y